# Day 10 - Finding the message

This may look daunting, but it's really quite simple if you look at just the `y` coordinates and their velocity. The numbers are really big but the velocities are relatively small, 5 at most. For these to form letters the ones with the same y velocity must all *already* be within a small range, the letters have a limited height even if arranged in multiple rows.

So you can look at the band of values with the same velocity; the minimum and maximum values of these will fall into a small range. Do the same for the opposite end of the y axis, with the velocity negated. All you need to do is get those two ranges to overlap in a single second.

You can then find the $t$ time component; if $max(y^v)$ is the highest value in the positive velocity $v$ band, and $max(y^{-v})$ is the corresponding value for the negative velocity $-v$, then the formula should roughly be $max(y^v) + (vt) = max(y^{-v}) + (-vt)$, give or take a few stars overlap, so `t` can be extracted as

$$t = \lfloor \frac{max(y^{-v}) - max(y^v)}{2v} \rfloor$$

where the $\lfloor \cdots \rfloor$ notation is the integer floor value of the division. As it turns out, the value of `t` is the answer for part two.

In [1]:
import aocd

data = aocd.get_data(day=10, year=2018)

In [2]:
import numpy as np
from io import StringIO

# read into a series of [x, y, dx, dy] values
stars = np.fromregex(
    StringIO(data),
    r'position=<\s*(-?\d+),\s*(-?\d+)> velocity=<\s*(-?\d+),\s*(-?\d+)>',
    np.dtype('i4')
)

In [3]:
from PIL import Image

def extract_message(stars, scale=3):
    # symbolic names for column selection
    x, y, dx, dy = range(4)
    
    # determine time t
    v = stars[:, dy].max()
    posv, negv = stars[:, dy] == v, stars[:, dy] == -v
    maxy_posv, maxy_negv = stars[posv][:, y].max(), stars[negv][:, y].max()
    t = (maxy_negv - maxy_posv) // (2 * v)
    
    # move all x y positions by t * dx, t * dy to form coordinates
    coords = stars[:, [x, y]] + (t * stars[:, [dx, dy]])
    # normalise these to start at 0, 0
    minxy = coords[:, [x, y]].min(axis=0)
    normalised = coords - minxy
    
    # create an image matrix with value 255, white background in a PIL L-band image
    message = np.full(normalised.max(axis=0) + 1, 255)
    # Set the message points to 0, black
    message[normalised[:, x], normalised[:, y]] = 0
    # convert to a PIL image; transposing x and y to match PIL expectations
    image = Image.fromarray(message.T.astype(np.dtype('B')))
    return t, image.resize((image.size[0] * scale, image.size[1] * scale))

In [4]:
from IPython.display import display
time_taken, message = extract_message(stars)
print('Part 1:')
display(message)
print('Part 2:', time_taken)

Part 1:


Part 2: 10659
